# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Load Files

In [2]:
outputs_dir = Path("outputs")
res_fp = outputs_dir / "valuation_results_3y.csv"
scen_fp = outputs_dir / "valuation_scenarios.csv"
sens_fp = outputs_dir / "valuation_sensitivity_long.csv"

missing = [p.name for p in [res_fp, scen_fp, sens_fp] if not p.exists()]
if missing:
    raise FileNotFoundError(f"필수 결과 파일 누락: {missing}. 먼저 3년 정상화/시나리오/민감도 파일을 생성해 주세요.")

out = pd.read_csv(res_fp, dtype={"Symbol":"string"})
scen = pd.read_csv(scen_fp, dtype={"Symbol":"string"})
sens = pd.read_csv(sens_fp, dtype={"Symbol":"string"})

# 1) Derive Market Price Proxy (from PER or PBR)

In [3]:
# Prefer PBR route if book equity is available and shares>0
def derive_market_price(row):
    price_pbr = np.nan
    price_per = np.nan
    try:
        shares = float(row.get("기말발행주식수", np.nan))
        ni = float(row.get("당기순이익", np.nan))
        per = float(row.get("PER", np.nan))
        pbr = float(row.get("PBR", np.nan))
        total_assets = float(row.get("총자산", np.nan))
        eq_ratio = float(row.get("자기자본비율", np.nan))
    except Exception:
        shares = ni = per = pbr = total_assets = eq_ratio = np.nan

    # PBR-based
    if np.isfinite(pbr) and np.isfinite(total_assets) and np.isfinite(eq_ratio) and np.isfinite(shares) and shares>0 and eq_ratio>0:
        book_equity = total_assets * eq_ratio
        bvps = book_equity / shares
        price_pbr = pbr * bvps

    # PER-based
    if np.isfinite(per) and np.isfinite(ni) and np.isfinite(shares) and shares>0:
        eps = ni / shares
        price_per = per * eps

    # Choose: prefer PBR (usually more stable for banks/financials), else PER
    if np.isfinite(price_pbr):
        return float(price_pbr)
    elif np.isfinite(price_per):
        return float(price_per)
    else:
        return np.nan


In [4]:
out["MarketPriceProxy"] = out.apply(derive_market_price, axis=1)

# Filter to only rows with a usable market price
work = out.copy()

# 2) Relative screen (cheapness)

In [5]:
cheap_cols = [c for c in ["PER_cheap_3y","PBR_cheap_3y","EV_EBITDA_cheap_3y"] if c in work.columns]
work["cheap_mean_3y"] = np.nanmean(work[cheap_cols].values, axis=1)

C:\Users\PC\AppData\Local\Temp\ipykernel_22212\3733577130.py:2: RuntimeWarning: Mean of empty slice
  work["cheap_mean_3y"] = np.nanmean(work[cheap_cols].values, axis=1)


# 3) Absolute cross-check vs market (Base scenario using RIM first, fallback to DDM)

In [6]:
def choose_base_model_price(row):
    # Prefer RIM if present, else DDM
    rim = row.get("RIM_price", np.nan)
    ddm = row.get("DDM_price", np.nan)
    if np.isfinite(rim):
        return float(rim)
    elif np.isfinite(ddm):
        return float(ddm)
    else:
        return np.nan

work["Base_intrinsic"] = work.apply(choose_base_model_price, axis=1)
work["undervaluation_base"] = (work["Base_intrinsic"] / work["MarketPriceProxy"]) - 1.0


# 4) Scenario linkage: pull Conservative/Base/Optimistic (RIM-first, fallback DDM)

In [7]:
# Merge a pivot of scenarios for convenience
def scen_price_pref(group):
    # For each scenario, pick RIM_price if available else DDM_price
    m = {}
    for _, r in group.iterrows():
        scen = r["Scenario"]
        rim_p = r.get("RIM_price", np.nan)
        ddm_p = r.get("DDM_price", np.nan)
        val = rim_p if np.isfinite(rim_p) else ddm_p
        m[scen] = val
    return pd.Series(m)

scen_pivot = scen.groupby(["Symbol","Name"]).apply(scen_price_pref).reset_index()
work = work.merge(scen_pivot, on=["Symbol","Name"], how="left", suffixes=("",""))


In [8]:
# conservative/base/optimistic undervaluation
for scen_name in ["Conservative","Base","Optimistic"]:
    if scen_name in work.columns:
        work[f"undervaluation_{scen_name}"] = (work[scen_name] / work["MarketPriceProxy"]) - 1.0

# 5) Robustness from sensitivity grid
- For each symbol & model, fraction of grid points where Price >= MarketPriceProxy
  
- Combine DDM/RIM by averaging their pass rates (if both present)

In [9]:
sens = sens.merge(work[["Symbol","MarketPriceProxy"]], on="Symbol", how="left")
sens["pass"] = sens["Price"] >= sens["MarketPriceProxy"]

pass_rates = sens.groupby(["Symbol","Model"])["pass"].mean().reset_index().pivot(index="Symbol", columns="Model", values="pass")
pass_rates = pass_rates.reset_index().rename_axis(None, axis=1)
# Average across models if both exist
def avg_pass(row):
    vals = []
    for m in ["DDM","RIM"]:
        v = row.get(m, np.nan)
        if np.isfinite(v): vals.append(float(v))
    if len(vals)==0: return np.nan
    return float(np.mean(vals))

pass_rates["robust_score"] = pass_rates.apply(avg_pass, axis=1)
work = work.merge(pass_rates[["Symbol","robust_score"]], on="Symbol", how="left")


# 6) Final score & Recommendations
final_score = 0.5*cheap_mean_3y + 0.3*undervaluation_base + 0.2*robust_score

(clip to avoid nan spillovers)

In [10]:
def safe_num(x): 
    try:
        return float(x)
    except Exception:
        return np.nan

for c in ["cheap_mean_3y","undervaluation_base","robust_score"]:
    work[c] = work[c].astype(float)

work["final_score"] = (
    0.5*work["cheap_mean_3y"].fillna(0) + 
    0.3*work["undervaluation_base"].fillna(0) +
    0.2*work["robust_score"].fillna(0)
)

# Rank and select
work_sorted = work.sort_values("final_score", ascending=False)

top20 = work_sorted.head(20).copy()
bottom20 = work_sorted.tail(20).copy()

In [11]:
# Select columns for report
report_cols = [
    "Symbol","Name","산업분류(중분류)",
    "cheap_mean_3y",
    "MarketPriceProxy","Base_intrinsic","undervaluation_base",
    "Conservative","Base","Optimistic",
    "undervaluation_Conservative","undervaluation_Base","undervaluation_Optimistic",
    "robust_score","final_score"
]
report_cols = [c for c in report_cols if c in work.columns]

rec_fp_all = outputs_dir / "recommendations_scored.csv"
rec_fp_top = outputs_dir / "recommend_top20.csv"
rec_fp_bot = outputs_dir / "recommend_bottom20.csv"

work_sorted[report_cols].to_csv(rec_fp_all, index=False, encoding="utf-8-sig")
top20[report_cols].to_csv(rec_fp_top, index=False, encoding="utf-8-sig")
bottom20[report_cols].to_csv(rec_fp_bot, index=False, encoding="utf-8-sig")